<a href="https://colab.research.google.com/github/waltermayor/lidarscripts/blob/master/Ejemplo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


!pip install laspy

from google.colab import drive
drive.mount('/content/drive')

import laspy
import scipy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn import preprocessing
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import path

import cv2
import os

path='/content/drive/My Drive/projectUK/'
os.chdir(path)
!pwd

# graficar imagenes estaticas en el notebook
%matplotlib inline

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/projectUK


# **cargar nube de puntos**

In [2]:
# Open a file in read mode:
inFile = laspy.file.File('LAS/1.las')
# Grab a numpy dataset of our clustering dimensions:
dataset = np.vstack([inFile.x, inFile.y, inFile.z]).transpose()
dataset.shape

(1311463, 3)

# **filtrar en z**

In [3]:
%%time
def frange(start, stop, step):
  i = start
  while i < stop:
    yield i
    i += step
#ground points grid filter
n = 100 #grid step
dataset_Z_filtered = dataset[[0]]
zfiltered = (dataset[:, 2].max() - dataset[:, 2].min())/10 #setting height filtered from ground
print('zfiltered =', zfiltered)
xstep = (dataset[:, 0].max() - dataset[:, 0].min())/n
ystep = (dataset[:, 1].max() - dataset[:, 1].min())/n
for x in frange (dataset[:, 0].min(), dataset[:, 0].max(), xstep):
  for y in frange (dataset[:, 1].min(), dataset[:, 1].max(), ystep):
    datasetfiltered = dataset[(dataset[:,0] > x)
                             &(dataset[:, 0] < x+xstep)
                             &(dataset[:, 1] > y)
                             &(dataset[:, 1] < y+ystep)]
    if datasetfiltered.shape[0] > 0:
      datasetfiltered = datasetfiltered[datasetfiltered[:, 2]
                        >(datasetfiltered[:, 2].min()+ zfiltered)]
      if datasetfiltered.shape[0] > 0:
        dataset_Z_filtered = np.concatenate((dataset_Z_filtered,
                                             datasetfiltered))
print('dataset_Z_filtered shape', dataset_Z_filtered.shape)

zfiltered = 2.242
dataset_Z_filtered shape (54080, 3)
CPU times: user 1min 29s, sys: 4.62 s, total: 1min 33s
Wall time: 1min 33s


In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

# **mostrar tipos de datos**

In [4]:
print('Examining Point Format:' )
pointformat = inFile.point_format
for spec in inFile.point_format:
  print(spec.name)

Examining Point Format:
X
Y
Z
intensity
flag_byte
raw_classification
scan_angle_rank
user_data
pt_src_id


# **estadisticas de la base de datos**

In [5]:
print('Z range =', dataset[:, 2].max() - dataset[:, 2].min())
print('Z max =', dataset[:, 2].max(), 'Z min =', dataset[:, 2].min())
print('Y range =', dataset[:, 1].max() - dataset[:, 1].min())
print('Y max =', dataset[:, 1].max(), 'Y min =', dataset[:, 1].min())
print('X range =', dataset[:, 0].max() - dataset[:, 0].min())
print('X max =', dataset[:, 0].max(), 'X min =', dataset[:, 0].min())

Z range = 22.419999999999998
Z max = 20.31 Z min = -2.11
Y range = 1000.0
Y max = 296000.0 Y min = 295000.0
X range = 1000.0
X max = 571000.0 X min = 570000.0



# **normalizar dataset (datos con diferentes rangos)**



In [0]:
dataset = preprocessing.normalize(dataset)

# **clustering putnos cercanos**

In [0]:
clustering = DBSCAN(eps=2, min_samples=5, leaf_size=30).fit(dataset)

In [0]:
core_samples_mask = np.zeros_like(clustering.labels_, dtype=bool)
core_samples_mask[clustering.core_sample_indices_] = True
labels = clustering.labels_
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print('Estimated number of clusters: %d' % n_clusters_)
print('stimated number of noise points: %d' % n_noise_)

In [0]:
# Black removed and is used for noise instead.
fig = plt.figure(figsize=[100, 50])
ax = fig.add_subplot(111, projection='3d')
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          
for each in np.linspace(0, 1, len(unique_labels))]
  for k, col in zip(unique_labels, colors):
    if k == -1:
    # Black used for noise.
      col = [0, 0, 0, 1]
  class_member_mask = (labels == k)
  xyz = dataset[class_member_mask & core_samples_mask]
  ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2], c=col, marker=".")
plt.title('Estimated number of cluster: %d' % n_clusters_)
plt.show()